In [5]:
import nltk
import numpy as np
from datasets import load_dataset
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
import sys
import os
sys.path.append('../app')
from legal_summary import legal_summary_pipeline


## Downloading the Dataset here

In [19]:
multi_lexsum = load_dataset("allenai/multi_lexsum", name="v20230518", trust_remote_code=True)

In [20]:
train_df = multi_lexsum['train'].to_pandas()
val_df = multi_lexsum['validation'].to_pandas()
test_df = multi_lexsum['test'].to_pandas()
def extract_case_info(dataset_split):
    # Convert to pandas
    df = dataset_split.to_pandas()
    
    # Normalize the nested metadata
    meta_df = pd.json_normalize(df["case_metadata"])
    
    # Concatenate relevant columns
    combined_df = pd.concat([
        df["sources"],
        df["summary/long"],
        df["summary/short"],
        df["summary/tiny"],
        meta_df[["class_action_sought", "case_type"]]
    ], axis=1)
    
    return pd.DataFrame(combined_df.rename(columns={
        "sources": "full_case",
        "summary/long": "summary_long",
        "summary/short": "summary_short",
        "summary/tiny": "summary_tiny"
    }))

# Apply to train/val/test
train_cases_df = extract_case_info(multi_lexsum['train'])
val_cases_df = extract_case_info(multi_lexsum['validation'])
test_cases_df = extract_case_info(multi_lexsum['test'])


In [23]:

your_legal_text = train_cases_df.iloc[range(0,64), :]
temp_tiny = legal_summary_pipeline(train_cases_df, summary_style="tiny", batch_size = 16)

Summarizing cases (batch size: 16):   0%|                                                      | 0/199 [00:00<?, ?it/s]C:\Users\hongk\anaconda3\Lib\site-packages\transformers\models\bart\modeling_bart.py:597: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Summarizing cases (batch size: 16): 100%|██████████████████████████████████████████| 199/199 [5:30:48<00:00, 99.74s/it]


In [25]:
temp_tiny.to_csv('Legal_Summarized_Case_Tiny.csv', index = False)